# MoA

Fork from [here](https://www.kaggle.com/simakov/keras-multilabel-neural-network-v1-2)

In [1]:
import sys

sys.path.append("../input/iterative-stratification/iterative-stratification-master")
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [2]:
import os
import random

import numpy as np
import pandas as pd

# import optuna
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as L
import tensorflow_addons as tfa
from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import log_loss
from sklearn.preprocessing import QuantileTransformer
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

## 関数

In [3]:
def fix_seed(seed=2020):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)


random_seed = 22
fix_seed(random_seed)

### Label Smoothing

https://www.kaggle.com/rahulsd91/moa-label-smoothing?scriptVersionId=44383706



In [4]:
# Prediction Clipping Thresholds
p_min = 0.001
p_max = 0.999

# Evaluation Metric with clipping and no label smoothing
def metric(y_true, y_pred):
    y_pred = tf.clip_by_value(y_pred, p_min, p_max)
    return -K.mean(y_true * K.log(y_pred) + (1 - y_true) * K.log(1 - y_pred))

## データロード

In [5]:
train_df = pd.read_csv("../input/lish-moa/train_features.csv")
test_df = pd.read_csv("../input/lish-moa/test_features.csv")
target_df = pd.read_csv("../input/lish-moa/train_targets_scored.csv")
non_target_df = pd.read_csv("../input/lish-moa/train_targets_nonscored.csv")
submit_df = pd.read_csv("../input/lish-moa/sample_submission.csv")

In [6]:
train = train_df.copy()
test = test_df.copy()

## 前処理

In [7]:
train.loc[:, "cp_dose"] = train.loc[:, "cp_dose"].map({"D1": 0, "D2": 1})
test.loc[:, "cp_dose"] = test.loc[:, "cp_dose"].map({"D1": 0, "D2": 1})

### cp_type が ctrl_vehicle なものは MoA を持たない

ので、学習から除外する

In [8]:
target_df = target_df.loc[train["cp_type"] != "ctl_vehicle"].reset_index(drop=True)
train = train.loc[train["cp_type"] != "ctl_vehicle"].reset_index(drop=True)

In [9]:
train = train.drop("cp_type", axis=1)
test = test.drop("cp_type", axis=1)

In [10]:
del train["sig_id"]
del target_df["sig_id"]
del test["sig_id"]

In [11]:
train

,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,g-7,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,24,0,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,-1.0220,-0.0326,...,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
1,72,0,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,0.2341,0.3372,...,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371
2,48,0,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,0.1715,0.2155,...,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931
3,48,0,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,-1.9590,0.1792,...,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154
4,72,1,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,-0.2800,-0.1498,...,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21943,72,0,0.1608,-1.0500,0.2551,-0.2239,-0.2431,0.4256,-0.1166,-0.1777,...,0.0789,0.3538,0.0558,0.3377,-0.4753,-0.2504,-0.7415,0.8413,-0.4259,0.2434
21944,24,1,0.1394,-0.0636,-0.1112,-0.5080,-0.4713,0.7201,0.5773,0.3055,...,0.1969,0.0262,-0.8121,0.3434,0.5372,-0.3246,0.0631,0.9171,0.5258,0.4680
21945,24,1,-1.3260,0.3478,-0.3743,0.9905,-0.7178,0.6621,-0.2252,-0.5565,...,0.4286,0.4426,0.0423,-0.3195,-0.8086,-0.9798,-0.2084,-0.1224,-0.2715,0.3689
21946,24,0,0.6660,0.2324,0.4392,0.2044,0.8531,-0.0343,0.0323,0.0463,...,-0.1105,0.4258,-0.2012,0.1506,1.5230,0.7101,0.1732,0.7015,-0.6290,0.0740


### Rank Gauss

https://www.kaggle.com/nayuts/moa-pytorch-nn-pca-rankgauss

連続値を特定の範囲の閉域に押し込めて、分布の偏りを解消する方法です。

In [12]:
g_cols = [col for col in train_df.columns if col.startswith("g-")]
c_cols = [col for col in train_df.columns if col.startswith("c-")]

for col in g_cols + c_cols:
    transformer = QuantileTransformer(n_quantiles=100, random_state=random_seed, output_distribution="normal")

    vec_len = len(train[col].values)
    vec_len_test = len(test[col].values)

    raw_vec = train[col].values.reshape(vec_len, 1)
    transformer.fit(raw_vec)

    train[col] = transformer.transform(raw_vec).reshape(1, vec_len)[0]
    test[col] = transformer.transform(test[col].values.reshape(vec_len_test, 1)).reshape(1, vec_len_test)[0]

In [13]:
train

,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,g-7,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,24,0,1.111801,0.903367,-0.433829,-0.971728,-0.286559,-1.011388,-1.357431,-0.041716,...,0.435228,0.388106,1.297345,0.882752,-0.202495,1.052112,-0.472513,0.345458,0.591507,0.692516
1,72,0,0.105667,0.672509,0.257486,0.086759,1.199685,0.691813,0.353695,0.558374,...,-0.491941,1.148246,0.728406,0.097171,0.454821,0.773468,0.233309,0.207813,0.964312,1.223121
2,48,0,0.767036,0.942499,1.408911,-0.126492,-0.028694,1.490985,0.272541,0.359490,...,-0.794302,-0.715229,0.962055,0.096127,-1.176291,-0.361225,-0.727620,-0.248613,-1.076346,1.142699
3,48,0,-0.755626,-0.297077,-0.455058,0.765972,2.343522,-0.852713,-2.316440,0.301512,...,-1.381920,-0.730154,-1.612183,-1.211000,-0.911943,-1.191839,-1.286279,-0.943448,-0.439482,-0.881278
4,72,1,-0.468806,-0.504196,0.956769,0.975864,1.447729,-0.863807,-0.346926,-0.227072,...,0.045906,0.023813,1.057944,1.737007,0.844923,-0.344784,0.176914,0.457388,-0.428668,1.176713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21943,72,0,0.225332,-1.269833,0.203082,-0.361744,-0.365285,0.574150,-0.118761,-0.271980,...,0.144679,0.531217,0.104134,0.547313,-0.547039,-0.278963,-0.822632,1.339243,-0.480911,0.419996
21944,24,1,0.195938,-0.039550,-0.255311,-0.799750,-0.721089,0.924354,0.779755,0.509043,...,0.303342,0.053359,-0.846121,0.555644,0.821718,-0.369418,0.114555,1.470674,0.826858,0.772835
21945,24,1,-1.946077,0.575730,-0.604782,1.298848,-1.057997,0.856044,-0.274794,-0.731116,...,0.655890,0.665706,0.084499,-0.350560,-0.865833,-1.005158,-0.245946,-0.123235,-0.310856,0.613841
21946,24,0,0.803911,0.405671,0.418646,0.309494,1.068126,-0.020533,0.084007,0.087675,...,-0.105613,0.641177,-0.236051,0.272655,2.254169,1.132558,0.263331,1.109534,-0.658578,0.173880


### PCA features + Existing features

In [14]:
# g-
n_comp = 50

data = pd.concat([pd.DataFrame(train[g_cols]), pd.DataFrame(test[g_cols])])
data2 = PCA(n_components=n_comp, random_state=random_seed).fit_transform(data[g_cols])
train2 = data2[: train.shape[0]]
test2 = data2[-test.shape[0] :]

train2 = pd.DataFrame(train2, columns=[f"pca_G-{i}" for i in range(n_comp)])
test2 = pd.DataFrame(test2, columns=[f"pca_G-{i}" for i in range(n_comp)])

train = pd.concat((train, train2), axis=1)
test = pd.concat((test, test2), axis=1)

In [15]:
# c-
n_comp = 15

data = pd.concat([pd.DataFrame(train[c_cols]), pd.DataFrame(test[c_cols])])
data2 = PCA(n_components=n_comp, random_state=random_seed).fit_transform(data[c_cols])
train2 = data2[: train.shape[0]]
test2 = data2[-test.shape[0] :]

train2 = pd.DataFrame(train2, columns=[f"pca_C-{i}" for i in range(n_comp)])
test2 = pd.DataFrame(test2, columns=[f"pca_C-{i}" for i in range(n_comp)])

train = pd.concat((train, train2), axis=1)
test = pd.concat((test, test2), axis=1)

In [16]:
train

,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,g-7,...,pca_C-5,pca_C-6,pca_C-7,pca_C-8,pca_C-9,pca_C-10,pca_C-11,pca_C-12,pca_C-13,pca_C-14
0,24,0,1.111801,0.903367,-0.433829,-0.971728,-0.286559,-1.011388,-1.357431,-0.041716,...,1.128005,0.425778,-0.343062,-0.195748,0.352685,0.403829,0.277811,0.330617,-0.981467,0.679548
1,72,0,0.105667,0.672509,0.257486,0.086759,1.199685,0.691813,0.353695,0.558374,...,-0.676575,0.021277,-0.388366,-1.150616,0.137138,0.875934,0.165417,-0.475121,-1.175414,-0.739302
2,48,0,0.767036,0.942499,1.408911,-0.126492,-0.028694,1.490985,0.272541,0.359490,...,0.925477,-0.989601,-0.858368,-0.691935,-0.550415,0.581527,0.051508,0.303663,0.545320,-0.118935
3,48,0,-0.755626,-0.297077,-0.455058,0.765972,2.343522,-0.852713,-2.316440,0.301512,...,1.010039,0.635626,-0.677750,-1.010033,0.952495,2.045002,1.416006,-0.397999,-0.616688,0.142138
4,72,1,-0.468806,-0.504196,0.956769,0.975864,1.447729,-0.863807,-0.346926,-0.227072,...,-0.263976,-0.560569,-0.483119,0.088108,-0.381305,0.687056,-0.857168,0.024852,-0.463927,0.182058
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21943,72,0,0.225332,-1.269833,0.203082,-0.361744,-0.365285,0.574150,-0.118761,-0.271980,...,-0.529939,1.351794,-0.019124,-0.352074,-0.098632,0.090319,-0.409949,1.080525,-0.232746,0.303632
21944,24,1,0.195938,-0.039550,-0.255311,-0.799750,-0.721089,0.924354,0.779755,0.509043,...,-0.697863,0.200118,1.212996,-1.002997,-1.273071,0.963687,0.770501,0.796243,-0.197498,-1.433636
21945,24,1,-1.946077,0.575730,-0.604782,1.298848,-1.057997,0.856044,-0.274794,-0.731116,...,-1.749574,-1.653004,0.925076,0.404705,-0.663696,0.914806,0.134995,-1.697094,0.385734,0.509971
21946,24,0,0.803911,0.405671,0.418646,0.309494,1.068126,-0.020533,0.084007,0.087675,...,-0.492358,0.445483,0.295951,0.073333,1.863752,0.101821,-2.197211,0.720895,-0.241714,-0.673445


### feature Selection using Variance Encoding

分散がしきい値以下の特徴量を捨てます。

In [17]:
var_thresh = VarianceThreshold(threshold=0.5)

data = train.append(test)
data_transformed = var_thresh.fit_transform(data.iloc[:, 2:])

train_transformed = data_transformed[: train.shape[0]]
test_transformed = data_transformed[-test.shape[0] :]


train = pd.DataFrame(train[["cp_time", "cp_dose"]].values.reshape(-1, 2), columns=["cp_time", "cp_dose"])
train = pd.concat([train, pd.DataFrame(train_transformed)], axis=1)


test = pd.DataFrame(test[["cp_time", "cp_dose"]].values.reshape(-1, 2), columns=["cp_time", "cp_dose"])
test = pd.concat([test, pd.DataFrame(test_transformed)], axis=1)

In [18]:
train

,cp_time,cp_dose,0,1,2,3,4,5,6,7,...,927,928,929,930,931,932,933,934,935,936
0,24,0,1.111801,0.903367,-0.433829,-0.971728,-0.286559,-1.011388,-1.357431,-0.041716,...,1.128005,0.425778,-0.343062,-0.195748,0.352685,0.403829,0.277811,0.330617,-0.981467,0.679548
1,72,0,0.105667,0.672509,0.257486,0.086759,1.199685,0.691813,0.353695,0.558374,...,-0.676575,0.021277,-0.388366,-1.150616,0.137138,0.875934,0.165417,-0.475121,-1.175414,-0.739302
2,48,0,0.767036,0.942499,1.408911,-0.126492,-0.028694,1.490985,0.272541,0.359490,...,0.925477,-0.989601,-0.858368,-0.691935,-0.550415,0.581527,0.051508,0.303663,0.545320,-0.118935
3,48,0,-0.755626,-0.297077,-0.455058,0.765972,2.343522,-0.852713,-2.316440,0.301512,...,1.010039,0.635626,-0.677750,-1.010033,0.952495,2.045002,1.416006,-0.397999,-0.616688,0.142138
4,72,1,-0.468806,-0.504196,0.956769,0.975864,1.447729,-0.863807,-0.346926,-0.227072,...,-0.263976,-0.560569,-0.483119,0.088108,-0.381305,0.687056,-0.857168,0.024852,-0.463927,0.182058
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21943,72,0,0.225332,-1.269833,0.203082,-0.361744,-0.365285,0.574150,-0.118761,-0.271980,...,-0.529939,1.351794,-0.019124,-0.352074,-0.098632,0.090319,-0.409949,1.080525,-0.232746,0.303632
21944,24,1,0.195938,-0.039550,-0.255311,-0.799750,-0.721089,0.924354,0.779755,0.509043,...,-0.697863,0.200118,1.212996,-1.002997,-1.273071,0.963687,0.770501,0.796243,-0.197498,-1.433636
21945,24,1,-1.946077,0.575730,-0.604782,1.298848,-1.057997,0.856044,-0.274794,-0.731116,...,-1.749574,-1.653004,0.925076,0.404705,-0.663696,0.914806,0.134995,-1.697094,0.385734,0.509971
21946,24,0,0.803911,0.405671,0.418646,0.309494,1.068126,-0.020533,0.084007,0.087675,...,-0.492358,0.445483,0.295951,0.073333,1.863752,0.101821,-2.197211,0.720895,-0.241714,-0.673445


## モデル作成

In [19]:
def create_model():
    model = tf.keras.Sequential(
        [
            L.Input(len(train.columns)),
            L.BatchNormalization(),
            tfa.layers.WeightNormalization(L.Dense(2048, activation="relu")),
            L.BatchNormalization(),
            L.Dropout(0.4),
            tfa.layers.WeightNormalization(L.Dense(2048, activation="relu")),
            L.BatchNormalization(),
            L.Dropout(0.4),
            tfa.layers.WeightNormalization(L.Dense(206, activation="sigmoid")),
        ]
    )
    model.compile(
        optimizer=tfa.optimizers.Lookahead(tf.optimizers.Adam(), sync_period=10),
        loss="binary_crossentropy",
    )
    return model

### TabNet for Tensorflow for MultiLabel

https://www.kaggle.com/gogo827jz/moa-stacked-tabnet-baseline-tensorflow-2-0#Model-Functions

In [25]:
def register_keras_custom_object(cls):
    tf.keras.utils.get_custom_objects()[cls.__name__] = cls
    return cls

In [26]:
def glu(x, n_units=None):
    """Generalized linear unit nonlinear activation."""
    if n_units is None:
        n_units = tf.shape(x)[-1] // 2

    return x[..., :n_units] * tf.nn.sigmoid(x[..., n_units:])

In [27]:
"""
Code replicated from https://github.com/tensorflow/addons/blob/master/tensorflow_addons/activations/sparsemax.py
"""


@register_keras_custom_object
@tf.function
def sparsemax(logits, axis):
    """Sparsemax activation function [1].
    For each batch `i` and class `j` we have
      $$sparsemax[i, j] = max(logits[i, j] - tau(logits[i, :]), 0)$$
    [1]: https://arxiv.org/abs/1602.02068
    Args:
        logits: Input tensor.
        axis: Integer, axis along which the sparsemax operation is applied.
    Returns:
        Tensor, output of sparsemax transformation. Has the same type and
        shape as `logits`.
    Raises:
        ValueError: In case `dim(logits) == 1`.
    """
    logits = tf.convert_to_tensor(logits, name="logits")

    # We need its original shape for shape inference.
    shape = logits.get_shape()
    rank = shape.rank
    is_last_axis = (axis == -1) or (axis == rank - 1)

    if is_last_axis:
        output = _compute_2d_sparsemax(logits)
        output.set_shape(shape)
        return output

    # If dim is not the last dimension, we have to do a transpose so that we can
    # still perform softmax on its last dimension.

    # Swap logits' dimension of dim and its last dimension.
    rank_op = tf.rank(logits)
    axis_norm = axis % rank
    logits = _swap_axis(logits, axis_norm, tf.math.subtract(rank_op, 1))

    # Do the actual softmax on its last dimension.
    output = _compute_2d_sparsemax(logits)
    output = _swap_axis(output, axis_norm, tf.math.subtract(rank_op, 1))

    # Make shape inference work since transpose may erase its static shape.
    output.set_shape(shape)
    return output

In [28]:
def _swap_axis(logits, dim_index, last_index, **kwargs):
    return tf.transpose(
        logits,
        tf.concat(
            [
                tf.range(dim_index),
                [last_index],
                tf.range(dim_index + 1, last_index),
                [dim_index],
            ],
            0,
        ),
        **kwargs,
    )

In [29]:
def _compute_2d_sparsemax(logits):
    """Performs the sparsemax operation when axis=-1."""
    shape_op = tf.shape(logits)
    obs = tf.math.reduce_prod(shape_op[:-1])
    dims = shape_op[-1]

    # In the paper, they call the logits z.
    # The mean(logits) can be substracted from logits to make the algorithm
    # more numerically stable. the instability in this algorithm comes mostly
    # from the z_cumsum. Substacting the mean will cause z_cumsum to be close
    # to zero. However, in practise the numerical instability issues are very
    # minor and substacting the mean causes extra issues with inf and nan
    # input.
    # Reshape to [obs, dims] as it is almost free and means the remanining
    # code doesn't need to worry about the rank.
    z = tf.reshape(logits, [obs, dims])

    # sort z
    z_sorted, _ = tf.nn.top_k(z, k=dims)

    # calculate k(z)
    z_cumsum = tf.math.cumsum(z_sorted, axis=-1)
    k = tf.range(1, tf.cast(dims, logits.dtype) + 1, dtype=logits.dtype)
    z_check = 1 + k * z_sorted > z_cumsum
    # because the z_check vector is always [1,1,...1,0,0,...0] finding the
    # (index + 1) of the last `1` is the same as just summing the number of 1.
    k_z = tf.math.reduce_sum(tf.cast(z_check, tf.int32), axis=-1)

    # calculate tau(z)
    # If there are inf values or all values are -inf, the k_z will be zero,
    # this is mathematically invalid and will also cause the gather_nd to fail.
    # Prevent this issue for now by setting k_z = 1 if k_z = 0, this is then
    # fixed later (see p_safe) by returning p = nan. This results in the same
    # behavior as softmax.
    k_z_safe = tf.math.maximum(k_z, 1)
    indices = tf.stack([tf.range(0, obs), tf.reshape(k_z_safe, [-1]) - 1], axis=1)
    tau_sum = tf.gather_nd(z_cumsum, indices)
    tau_z = (tau_sum - 1) / tf.cast(k_z, logits.dtype)

    # calculate p
    p = tf.math.maximum(tf.cast(0, logits.dtype), z - tf.expand_dims(tau_z, -1))
    # If k_z = 0 or if z = nan, then the input is invalid
    p_safe = tf.where(
        tf.expand_dims(
            tf.math.logical_or(tf.math.equal(k_z, 0), tf.math.is_nan(z_cumsum[:, -1])),
            axis=-1,
        ),
        tf.fill([obs, dims], tf.cast(float("nan"), logits.dtype)),
        p,
    )

    # Reshape back to original size
    p_safe = tf.reshape(p_safe, shape_op)
    return p_safe

In [30]:
"""
Code replicated from https://github.com/tensorflow/addons/blob/master/tensorflow_addons/layers/normalizations.py
"""


@register_keras_custom_object
class GroupNormalization(tf.keras.layers.Layer):
    """Group normalization layer.
    Group Normalization divides the channels into groups and computes
    within each group the mean and variance for normalization.
    Empirically, its accuracy is more stable than batch norm in a wide
    range of small batch sizes, if learning rate is adjusted linearly
    with batch sizes.
    Relation to Layer Normalization:
    If the number of groups is set to 1, then this operation becomes identical
    to Layer Normalization.
    Relation to Instance Normalization:
    If the number of groups is set to the
    input dimension (number of groups is equal
    to number of channels), then this operation becomes
    identical to Instance Normalization.
    Arguments
        groups: Integer, the number of groups for Group Normalization.
            Can be in the range [1, N] where N is the input dimension.
            The input dimension must be divisible by the number of groups.
        axis: Integer, the axis that should be normalized.
        epsilon: Small float added to variance to avoid dividing by zero.
        center: If True, add offset of `beta` to normalized tensor.
            If False, `beta` is ignored.
        scale: If True, multiply by `gamma`.
            If False, `gamma` is not used.
        beta_initializer: Initializer for the beta weight.
        gamma_initializer: Initializer for the gamma weight.
        beta_regularizer: Optional regularizer for the beta weight.
        gamma_regularizer: Optional regularizer for the gamma weight.
        beta_constraint: Optional constraint for the beta weight.
        gamma_constraint: Optional constraint for the gamma weight.
    Input shape
        Arbitrary. Use the keyword argument `input_shape`
        (tuple of integers, does not include the samples axis)
        when using this layer as the first layer in a model.
    Output shape
        Same shape as input.
    References
        - [Group Normalization](https://arxiv.org/abs/1803.08494)
    """

    def __init__(
        self,
        groups: int = 2,
        axis: int = -1,
        epsilon: float = 1e-3,
        center: bool = True,
        scale: bool = True,
        beta_initializer="zeros",
        gamma_initializer="ones",
        beta_regularizer=None,
        gamma_regularizer=None,
        beta_constraint=None,
        gamma_constraint=None,
        **kwargs
    ):
        super().__init__(**kwargs)
        self.supports_masking = True
        self.groups = groups
        self.axis = axis
        self.epsilon = epsilon
        self.center = center
        self.scale = scale
        self.beta_initializer = tf.keras.initializers.get(beta_initializer)
        self.gamma_initializer = tf.keras.initializers.get(gamma_initializer)
        self.beta_regularizer = tf.keras.regularizers.get(beta_regularizer)
        self.gamma_regularizer = tf.keras.regularizers.get(gamma_regularizer)
        self.beta_constraint = tf.keras.constraints.get(beta_constraint)
        self.gamma_constraint = tf.keras.constraints.get(gamma_constraint)
        self._check_axis()

    def build(self, input_shape):

        self._check_if_input_shape_is_none(input_shape)
        self._set_number_of_groups_for_instance_norm(input_shape)
        self._check_size_of_dimensions(input_shape)
        self._create_input_spec(input_shape)

        self._add_gamma_weight(input_shape)
        self._add_beta_weight(input_shape)
        self.built = True
        super().build(input_shape)

    def call(self, inputs, training=None):
        # Training=none is just for compat with batchnorm signature call
        input_shape = tf.keras.backend.int_shape(inputs)
        tensor_input_shape = tf.shape(inputs)

        reshaped_inputs, group_shape = self._reshape_into_groups(inputs, input_shape, tensor_input_shape)

        normalized_inputs = self._apply_normalization(reshaped_inputs, input_shape)

        outputs = tf.reshape(normalized_inputs, tensor_input_shape)

        return outputs

    def get_config(self):
        config = {
            "groups": self.groups,
            "axis": self.axis,
            "epsilon": self.epsilon,
            "center": self.center,
            "scale": self.scale,
            "beta_initializer": tf.keras.initializers.serialize(self.beta_initializer),
            "gamma_initializer": tf.keras.initializers.serialize(self.gamma_initializer),
            "beta_regularizer": tf.keras.regularizers.serialize(self.beta_regularizer),
            "gamma_regularizer": tf.keras.regularizers.serialize(self.gamma_regularizer),
            "beta_constraint": tf.keras.constraints.serialize(self.beta_constraint),
            "gamma_constraint": tf.keras.constraints.serialize(self.gamma_constraint),
        }
        base_config = super().get_config()
        return {**base_config, **config}

    def compute_output_shape(self, input_shape):
        return input_shape

    def _reshape_into_groups(self, inputs, input_shape, tensor_input_shape):

        group_shape = [tensor_input_shape[i] for i in range(len(input_shape))]
        group_shape[self.axis] = input_shape[self.axis] // self.groups
        group_shape.insert(self.axis, self.groups)
        group_shape = tf.stack(group_shape)
        reshaped_inputs = tf.reshape(inputs, group_shape)
        return reshaped_inputs, group_shape

    def _apply_normalization(self, reshaped_inputs, input_shape):

        group_shape = tf.keras.backend.int_shape(reshaped_inputs)
        group_reduction_axes = list(range(1, len(group_shape)))
        axis = -2 if self.axis == -1 else self.axis - 1
        group_reduction_axes.pop(axis)

        mean, variance = tf.nn.moments(reshaped_inputs, group_reduction_axes, keepdims=True)

        gamma, beta = self._get_reshaped_weights(input_shape)
        normalized_inputs = tf.nn.batch_normalization(
            reshaped_inputs,
            mean=mean,
            variance=variance,
            scale=gamma,
            offset=beta,
            variance_epsilon=self.epsilon,
        )
        return normalized_inputs

    def _get_reshaped_weights(self, input_shape):
        broadcast_shape = self._create_broadcast_shape(input_shape)
        gamma = None
        beta = None
        if self.scale:
            gamma = tf.reshape(self.gamma, broadcast_shape)

        if self.center:
            beta = tf.reshape(self.beta, broadcast_shape)
        return gamma, beta

    def _check_if_input_shape_is_none(self, input_shape):
        dim = input_shape[self.axis]
        if dim is None:
            raise ValueError(
                "Axis " + str(self.axis) + " of "
                "input tensor should have a defined dimension "
                "but the layer received an input with shape " + str(input_shape) + "."
            )

    def _set_number_of_groups_for_instance_norm(self, input_shape):
        dim = input_shape[self.axis]

        if self.groups == -1:
            self.groups = dim

    def _check_size_of_dimensions(self, input_shape):

        dim = input_shape[self.axis]
        if dim < self.groups:
            raise ValueError(
                "Number of groups (" + str(self.groups) + ") cannot be "
                "more than the number of channels (" + str(dim) + ")."
            )

        if dim % self.groups != 0:
            raise ValueError(
                "Number of groups (" + str(self.groups) + ") must be a "
                "multiple of the number of channels (" + str(dim) + ")."
            )

    def _check_axis(self):

        if self.axis == 0:
            raise ValueError(
                "You are trying to normalize your batch axis. Do you want to "
                "use tf.layer.batch_normalization instead"
            )

    def _create_input_spec(self, input_shape):

        dim = input_shape[self.axis]
        self.input_spec = tf.keras.layers.InputSpec(ndim=len(input_shape), axes={self.axis: dim})

    def _add_gamma_weight(self, input_shape):

        dim = input_shape[self.axis]
        shape = (dim,)

        if self.scale:
            self.gamma = self.add_weight(
                shape=shape,
                name="gamma",
                initializer=self.gamma_initializer,
                regularizer=self.gamma_regularizer,
                constraint=self.gamma_constraint,
            )
        else:
            self.gamma = None

    def _add_beta_weight(self, input_shape):

        dim = input_shape[self.axis]
        shape = (dim,)

        if self.center:
            self.beta = self.add_weight(
                shape=shape,
                name="beta",
                initializer=self.beta_initializer,
                regularizer=self.beta_regularizer,
                constraint=self.beta_constraint,
            )
        else:
            self.beta = None

    def _create_broadcast_shape(self, input_shape):
        broadcast_shape = [1] * len(input_shape)
        broadcast_shape[self.axis] = input_shape[self.axis] // self.groups
        broadcast_shape.insert(self.axis, self.groups)
        return broadcast_shape

In [31]:
class TransformBlock(tf.keras.Model):
    def __init__(self, features, norm_type, momentum=0.9, virtual_batch_size=None, groups=2, block_name="", **kwargs):
        super(TransformBlock, self).__init__(**kwargs)

        self.features = features
        self.norm_type = norm_type
        self.momentum = momentum
        self.groups = groups
        self.virtual_batch_size = virtual_batch_size

        self.transform = tf.keras.layers.Dense(self.features, use_bias=False, name=f"transformblock_dense_{block_name}")

        if norm_type == "batch":
            self.bn = tf.keras.layers.BatchNormalization(
                axis=-1,
                momentum=momentum,
                virtual_batch_size=virtual_batch_size,
                name=f"transformblock_bn_{block_name}",
            )

        else:
            self.bn = GroupNormalization(axis=-1, groups=self.groups, name=f"transformblock_gn_{block_name}")

    def call(self, inputs, training=None):
        x = self.transform(inputs)
        x = self.bn(x, training=training)
        return x

In [32]:
class TabNet(tf.keras.Model):
    def __init__(
        self,
        feature_columns,
        feature_dim=64,
        output_dim=64,
        num_features=None,
        num_decision_steps=5,
        relaxation_factor=1.5,
        sparsity_coefficient=1e-5,
        norm_type="group",
        batch_momentum=0.98,
        virtual_batch_size=None,
        num_groups=2,
        epsilon=1e-5,
        **kwargs,
    ):
        """
        Tensorflow 2.0 implementation of [TabNet: Attentive Interpretable Tabular Learning](https://arxiv.org/abs/1908.07442)
        # Hyper Parameter Tuning (Excerpt from the paper)
        We consider datasets ranging from ∼10K to ∼10M training points, with varying degrees of fitting
        difficulty. TabNet obtains high performance for all with a few general principles on hyperparameter
        selection:
            - Most datasets yield the best results for Nsteps ∈ [3, 10]. Typically, larger datasets and
            more complex tasks require a larger Nsteps. A very high value of Nsteps may suffer from
            overfitting and yield poor generalization.
            - Adjustment of the values of Nd and Na is the most efficient way of obtaining a trade-off
            between performance and complexity. Nd = Na is a reasonable choice for most datasets. A
            very high value of Nd and Na may suffer from overfitting and yield poor generalization.
            - An optimal choice of γ can have a major role on the overall performance. Typically a larger
            Nsteps value favors for a larger γ.
            - A large batch size is beneficial for performance - if the memory constraints permit, as large
            as 1-10 % of the total training dataset size is suggested. The virtual batch size is typically
            much smaller than the batch size.
            - Initially large learning rate is important, which should be gradually decayed until convergence.
        Args:
            feature_columns: The Tensorflow feature columns for the dataset.
            feature_dim (N_a): Dimensionality of the hidden representation in feature
                transformation block. Each layer first maps the representation to a
                2*feature_dim-dimensional output and half of it is used to determine the
                nonlinearity of the GLU activation where the other half is used as an
                input to GLU, and eventually feature_dim-dimensional output is
                transferred to the next layer.
            output_dim (N_d): Dimensionality of the outputs of each decision step, which is
                later mapped to the final classification or regression output.
            num_features: The number of input features (i.e the number of columns for
                tabular data assuming each feature is represented with 1 dimension).
            num_decision_steps(N_steps): Number of sequential decision steps.
            relaxation_factor (gamma): Relaxation factor that promotes the reuse of each
                feature at different decision steps. When it is 1, a feature is enforced
                to be used only at one decision step and as it increases, more
                flexibility is provided to use a feature at multiple decision steps.
            sparsity_coefficient (lambda_sparse): Strength of the sparsity regularization.
                Sparsity may provide a favorable inductive bias for convergence to
                higher accuracy for some datasets where most of the input features are redundant.
            norm_type: Type of normalization to perform for the model. Can be either
                'batch' or 'group'. 'group' is the default.
            batch_momentum: Momentum in ghost batch normalization.
            virtual_batch_size: Virtual batch size in ghost batch normalization. The
                overall batch size should be an integer multiple of virtual_batch_size.
            num_groups: Number of groups used for group normalization.
            epsilon: A small number for numerical stability of the entropy calculations.
        """
        super(TabNet, self).__init__(**kwargs)

        # Input checks
        if feature_columns is not None:
            if type(feature_columns) not in (list, tuple):
                raise ValueError("`feature_columns` must be a list or a tuple.")

            if len(feature_columns) == 0:
                raise ValueError("`feature_columns` must be contain at least 1 tf.feature_column !")

            if num_features is None:
                num_features = len(feature_columns)
            else:
                num_features = int(num_features)

        else:
            if num_features is None:
                raise ValueError("If `feature_columns` is None, then `num_features` cannot be None.")

        if num_decision_steps < 1:
            raise ValueError("Num decision steps must be greater than 0.")

        if feature_dim < output_dim:
            raise ValueError("To compute `features_for_coef`, feature_dim must be larger than output dim")

        feature_dim = int(feature_dim)
        output_dim = int(output_dim)
        num_decision_steps = int(num_decision_steps)
        relaxation_factor = float(relaxation_factor)
        sparsity_coefficient = float(sparsity_coefficient)
        batch_momentum = float(batch_momentum)
        num_groups = max(1, int(num_groups))
        epsilon = float(epsilon)

        if relaxation_factor < 0.0:
            raise ValueError("`relaxation_factor` cannot be negative !")

        if sparsity_coefficient < 0.0:
            raise ValueError("`sparsity_coefficient` cannot be negative !")

        if virtual_batch_size is not None:
            virtual_batch_size = int(virtual_batch_size)

        if norm_type not in ["batch", "group"]:
            raise ValueError("`norm_type` must be either `batch` or `group`")

        self.feature_columns = feature_columns
        self.num_features = num_features
        self.feature_dim = feature_dim
        self.output_dim = output_dim

        self.num_decision_steps = num_decision_steps
        self.relaxation_factor = relaxation_factor
        self.sparsity_coefficient = sparsity_coefficient
        self.norm_type = norm_type
        self.batch_momentum = batch_momentum
        self.virtual_batch_size = virtual_batch_size
        self.num_groups = num_groups
        self.epsilon = epsilon

        # if num_decision_steps > 1:
        # features_for_coeff = feature_dim - output_dim
        # print(f"[TabNet]: {features_for_coeff} features will be used for decision steps.")

        if self.feature_columns is not None:
            self.input_features = tf.keras.layers.DenseFeatures(feature_columns, trainable=True)

            if self.norm_type == "batch":
                self.input_bn = tf.keras.layers.BatchNormalization(axis=-1, momentum=batch_momentum, name="input_bn")
            else:
                self.input_bn = GroupNormalization(axis=-1, groups=self.num_groups, name="input_gn")

        else:
            self.input_features = None
            self.input_bn = None

        self.transform_f1 = TransformBlock(
            2 * self.feature_dim,
            self.norm_type,
            self.batch_momentum,
            self.virtual_batch_size,
            self.num_groups,
            block_name="f1",
        )

        self.transform_f2 = TransformBlock(
            2 * self.feature_dim,
            self.norm_type,
            self.batch_momentum,
            self.virtual_batch_size,
            self.num_groups,
            block_name="f2",
        )

        self.transform_f3_list = [
            TransformBlock(
                2 * self.feature_dim,
                self.norm_type,
                self.batch_momentum,
                self.virtual_batch_size,
                self.num_groups,
                block_name=f"f3_{i}",
            )
            for i in range(self.num_decision_steps)
        ]

        self.transform_f4_list = [
            TransformBlock(
                2 * self.feature_dim,
                self.norm_type,
                self.batch_momentum,
                self.virtual_batch_size,
                self.num_groups,
                block_name=f"f4_{i}",
            )
            for i in range(self.num_decision_steps)
        ]

        self.transform_coef_list = [
            TransformBlock(
                self.num_features,
                self.norm_type,
                self.batch_momentum,
                self.virtual_batch_size,
                self.num_groups,
                block_name=f"coef_{i}",
            )
            for i in range(self.num_decision_steps - 1)
        ]

        self._step_feature_selection_masks = None
        self._step_aggregate_feature_selection_mask = None

    def call(self, inputs, training=None):
        if self.input_features is not None:
            features = self.input_features(inputs)
            features = self.input_bn(features, training=training)

        else:
            features = inputs

        batch_size = tf.shape(features)[0]
        self._step_feature_selection_masks = []
        self._step_aggregate_feature_selection_mask = None

        # Initializes decision-step dependent variables.
        output_aggregated = tf.zeros([batch_size, self.output_dim])
        masked_features = features
        mask_values = tf.zeros([batch_size, self.num_features])
        aggregated_mask_values = tf.zeros([batch_size, self.num_features])
        complementary_aggregated_mask_values = tf.ones([batch_size, self.num_features])

        total_entropy = 0.0
        entropy_loss = 0.0

        for ni in range(self.num_decision_steps):
            # Feature transformer with two shared and two decision step dependent
            # blocks is used below.=
            transform_f1 = self.transform_f1(masked_features, training=training)
            transform_f1 = glu(transform_f1, self.feature_dim)

            transform_f2 = self.transform_f2(transform_f1, training=training)
            transform_f2 = (glu(transform_f2, self.feature_dim) + transform_f1) * tf.math.sqrt(0.5)

            transform_f3 = self.transform_f3_list[ni](transform_f2, training=training)
            transform_f3 = (glu(transform_f3, self.feature_dim) + transform_f2) * tf.math.sqrt(0.5)

            transform_f4 = self.transform_f4_list[ni](transform_f3, training=training)
            transform_f4 = (glu(transform_f4, self.feature_dim) + transform_f3) * tf.math.sqrt(0.5)

            if ni > 0 or self.num_decision_steps == 1:
                decision_out = tf.nn.relu(transform_f4[:, : self.output_dim])

                # Decision aggregation.
                output_aggregated += decision_out

                # Aggregated masks are used for visualization of the
                # feature importance attributes.
                scale_agg = tf.reduce_sum(decision_out, axis=1, keepdims=True)

                if self.num_decision_steps > 1:
                    scale_agg = scale_agg / tf.cast(self.num_decision_steps - 1, tf.float32)

                aggregated_mask_values += mask_values * scale_agg

            features_for_coef = transform_f4[:, self.output_dim :]

            if ni < (self.num_decision_steps - 1):
                # Determines the feature masks via linear and nonlinear
                # transformations, taking into account of aggregated feature use.
                mask_values = self.transform_coef_list[ni](features_for_coef, training=training)
                mask_values *= complementary_aggregated_mask_values
                mask_values = sparsemax(mask_values, axis=-1)

                # Relaxation factor controls the amount of reuse of features between
                # different decision blocks and updated with the values of
                # coefficients.
                complementary_aggregated_mask_values *= self.relaxation_factor - mask_values

                # Entropy is used to penalize the amount of sparsity in feature
                # selection.
                total_entropy += tf.reduce_mean(
                    tf.reduce_sum(-mask_values * tf.math.log(mask_values + self.epsilon), axis=1)
                ) / (tf.cast(self.num_decision_steps - 1, tf.float32))

                # Add entropy loss
                entropy_loss = total_entropy

                # Feature selection.
                masked_features = tf.multiply(mask_values, features)

                # Visualization of the feature selection mask at decision step ni
                # tf.summary.image(
                #     "Mask for step" + str(ni),
                #     tf.expand_dims(tf.expand_dims(mask_values, 0), 3),
                #     max_outputs=1)
                mask_at_step_i = tf.expand_dims(tf.expand_dims(mask_values, 0), 3)
                self._step_feature_selection_masks.append(mask_at_step_i)

            else:
                # This branch is needed for correct compilation by tf.autograph
                entropy_loss = 0.0

        # Adds the loss automatically
        self.add_loss(self.sparsity_coefficient * entropy_loss)

        # Visualization of the aggregated feature importances
        # tf.summary.image(
        #     "Aggregated mask",
        #     tf.expand_dims(tf.expand_dims(aggregated_mask_values, 0), 3),
        #     max_outputs=1)

        agg_mask = tf.expand_dims(tf.expand_dims(aggregated_mask_values, 0), 3)
        self._step_aggregate_feature_selection_mask = agg_mask

        return output_aggregated

    @property
    def feature_selection_masks(self):
        return self._step_feature_selection_masks

    @property
    def aggregate_feature_selection_mask(self):
        return self._step_aggregate_feature_selection_mask

In [33]:
class TabNetClassifier(tf.keras.Model):
    def __init__(
        self,
        feature_columns,
        num_classes,
        num_features=None,
        feature_dim=64,
        output_dim=64,
        num_decision_steps=5,
        relaxation_factor=1.5,
        sparsity_coefficient=1e-5,
        norm_type="group",
        batch_momentum=0.98,
        virtual_batch_size=None,
        num_groups=1,
        epsilon=1e-5,
        multi_label=False,
        **kwargs
    ):
        """
        Tensorflow 2.0 implementation of [TabNet: Attentive Interpretable Tabular Learning](https://arxiv.org/abs/1908.07442)
        # Hyper Parameter Tuning (Excerpt from the paper)
        We consider datasets ranging from ∼10K to ∼10M training points, with varying degrees of fitting
        difficulty. TabNet obtains high performance for all with a few general principles on hyperparameter
        selection:
            - Most datasets yield the best results for Nsteps ∈ [3, 10]. Typically, larger datasets and
            more complex tasks require a larger Nsteps. A very high value of Nsteps may suffer from
            overfitting and yield poor generalization.
            - Adjustment of the values of Nd and Na is the most efficient way of obtaining a trade-off
            between performance and complexity. Nd = Na is a reasonable choice for most datasets. A
            very high value of Nd and Na may suffer from overfitting and yield poor generalization.
            - An optimal choice of γ can have a major role on the overall performance. Typically a larger
            Nsteps value favors for a larger γ.
            - A large batch size is beneficial for performance - if the memory constraints permit, as large
            as 1-10 % of the total training dataset size is suggested. The virtual batch size is typically
            much smaller than the batch size.
            - Initially large learning rate is important, which should be gradually decayed until convergence.
        Args:
            feature_columns: The Tensorflow feature columns for the dataset.
            num_classes: Number of classes.
            feature_dim (N_a): Dimensionality of the hidden representation in feature
                transformation block. Each layer first maps the representation to a
                2*feature_dim-dimensional output and half of it is used to determine the
                nonlinearity of the GLU activation where the other half is used as an
                input to GLU, and eventually feature_dim-dimensional output is
                transferred to the next layer.
            output_dim (N_d): Dimensionality of the outputs of each decision step, which is
                later mapped to the final classification or regression output.
            num_features: The number of input features (i.e the number of columns for
                tabular data assuming each feature is represented with 1 dimension).
            num_decision_steps(N_steps): Number of sequential decision steps.
            relaxation_factor (gamma): Relaxation factor that promotes the reuse of each
                feature at different decision steps. When it is 1, a feature is enforced
                to be used only at one decision step and as it increases, more
                flexibility is provided to use a feature at multiple decision steps.
            sparsity_coefficient (lambda_sparse): Strength of the sparsity regularization.
                Sparsity may provide a favorable inductive bias for convergence to
                higher accuracy for some datasets where most of the input features are redundant.
            norm_type: Type of normalization to perform for the model. Can be either
                'group' or 'group'. 'group' is the default.
            batch_momentum: Momentum in ghost batch normalization.
            virtual_batch_size: Virtual batch size in ghost batch normalization. The
                overall batch size should be an integer multiple of virtual_batch_size.
            num_groups: Number of groups used for group normalization.
            epsilon: A small number for numerical stability of the entropy calculations.
        """
        super(TabNetClassifier, self).__init__(**kwargs)

        self.num_classes = num_classes

        self.tabnet = TabNet(
            feature_columns=feature_columns,
            num_features=num_features,
            feature_dim=feature_dim,
            output_dim=output_dim,
            num_decision_steps=num_decision_steps,
            relaxation_factor=relaxation_factor,
            sparsity_coefficient=sparsity_coefficient,
            norm_type=norm_type,
            batch_momentum=batch_momentum,
            virtual_batch_size=virtual_batch_size,
            num_groups=num_groups,
            epsilon=epsilon,
            **kwargs
        )

        if multi_label:

            self.clf = tf.keras.layers.Dense(num_classes, activation="sigmoid", use_bias=False, name="classifier")

        else:

            self.clf = tf.keras.layers.Dense(num_classes, activation="softmax", use_bias=False, name="classifier")

    def call(self, inputs, training=None):
        self.activations = self.tabnet(inputs, training=training)
        out = self.clf(self.activations)

        return out

    def summary(self, *super_args, **super_kwargs):
        super().summary(*super_args, **super_kwargs)
        self.tabnet.summary(*super_args, **super_kwargs)

In [34]:
class TabNetRegressor(tf.keras.Model):
    def __init__(
        self,
        feature_columns,
        num_regressors,
        num_features=None,
        feature_dim=64,
        output_dim=64,
        num_decision_steps=5,
        relaxation_factor=1.5,
        sparsity_coefficient=1e-5,
        norm_type="group",
        batch_momentum=0.98,
        virtual_batch_size=None,
        num_groups=1,
        epsilon=1e-5,
        **kwargs
    ):
        """
        Tensorflow 2.0 implementation of [TabNet: Attentive Interpretable Tabular Learning](https://arxiv.org/abs/1908.07442)
        # Hyper Parameter Tuning (Excerpt from the paper)
        We consider datasets ranging from ∼10K to ∼10M training points, with varying degrees of fitting
        difficulty. TabNet obtains high performance for all with a few general principles on hyperparameter
        selection:
            - Most datasets yield the best results for Nsteps ∈ [3, 10]. Typically, larger datasets and
            more complex tasks require a larger Nsteps. A very high value of Nsteps may suffer from
            overfitting and yield poor generalization.
            - Adjustment of the values of Nd and Na is the most efficient way of obtaining a trade-off
            between performance and complexity. Nd = Na is a reasonable choice for most datasets. A
            very high value of Nd and Na may suffer from overfitting and yield poor generalization.
            - An optimal choice of γ can have a major role on the overall performance. Typically a larger
            Nsteps value favors for a larger γ.
            - A large batch size is beneficial for performance - if the memory constraints permit, as large
            as 1-10 % of the total training dataset size is suggested. The virtual batch size is typically
            much smaller than the batch size.
            - Initially large learning rate is important, which should be gradually decayed until convergence.
        Args:
            feature_columns: The Tensorflow feature columns for the dataset.
            num_regressors: Number of regression variables.
            feature_dim (N_a): Dimensionality of the hidden representation in feature
                transformation block. Each layer first maps the representation to a
                2*feature_dim-dimensional output and half of it is used to determine the
                nonlinearity of the GLU activation where the other half is used as an
                input to GLU, and eventually feature_dim-dimensional output is
                transferred to the next layer.
            output_dim (N_d): Dimensionality of the outputs of each decision step, which is
                later mapped to the final classification or regression output.
            num_features: The number of input features (i.e the number of columns for
                tabular data assuming each feature is represented with 1 dimension).
            num_decision_steps(N_steps): Number of sequential decision steps.
            relaxation_factor (gamma): Relaxation factor that promotes the reuse of each
                feature at different decision steps. When it is 1, a feature is enforced
                to be used only at one decision step and as it increases, more
                flexibility is provided to use a feature at multiple decision steps.
            sparsity_coefficient (lambda_sparse): Strength of the sparsity regularization.
                Sparsity may provide a favorable inductive bias for convergence to
                higher accuracy for some datasets where most of the input features are redundant.
            norm_type: Type of normalization to perform for the model. Can be either
                'group' or 'group'. 'group' is the default.
            batch_momentum: Momentum in ghost batch normalization.
            virtual_batch_size: Virtual batch size in ghost batch normalization. The
                overall batch size should be an integer multiple of virtual_batch_size.
            num_groups: Number of groups used for group normalization.
            epsilon: A small number for numerical stability of the entropy calculations.
        """
        super(TabNetRegressor, self).__init__(**kwargs)

        self.num_regressors = num_regressors

        self.tabnet = TabNet(
            feature_columns=feature_columns,
            num_features=num_features,
            feature_dim=feature_dim,
            output_dim=output_dim,
            num_decision_steps=num_decision_steps,
            relaxation_factor=relaxation_factor,
            sparsity_coefficient=sparsity_coefficient,
            norm_type=norm_type,
            batch_momentum=batch_momentum,
            virtual_batch_size=virtual_batch_size,
            num_groups=num_groups,
            epsilon=epsilon,
            **kwargs
        )

        self.regressor = tf.keras.layers.Dense(num_regressors, use_bias=False, name="regressor")

    def call(self, inputs, training=None):
        self.activations = self.tabnet(inputs, training=training)
        out = self.regressor(self.activations)
        return out

    def summary(self, *super_args, **super_kwargs):
        super().summary(*super_args, **super_kwargs)
        self.tabnet.summary(*super_args, **super_kwargs)

In [35]:
# Aliases
TabNetClassification = TabNetClassifier
TabNetRegression = TabNetRegressor

In [36]:
class StackedTabNet(tf.keras.Model):
    def __init__(
        self,
        feature_columns,
        num_layers=1,
        feature_dim=64,
        output_dim=64,
        num_features=None,
        num_decision_steps=5,
        relaxation_factor=1.5,
        sparsity_coefficient=1e-5,
        norm_type="group",
        batch_momentum=0.98,
        virtual_batch_size=None,
        num_groups=2,
        epsilon=1e-5,
        **kwargs
    ):
        """
        Tensorflow 2.0 implementation of [TabNet: Attentive Interpretable Tabular Learning](https://arxiv.org/abs/1908.07442)
        Stacked variant of the TabNet model, which stacks multiple TabNets into a singular model.
        # Hyper Parameter Tuning (Excerpt from the paper)
        We consider datasets ranging from ∼10K to ∼10M training points, with varying degrees of fitting
        difficulty. TabNet obtains high performance for all with a few general principles on hyperparameter
        selection:
            - Most datasets yield the best results for Nsteps ∈ [3, 10]. Typically, larger datasets and
            more complex tasks require a larger Nsteps. A very high value of Nsteps may suffer from
            overfitting and yield poor generalization.
            - Adjustment of the values of Nd and Na is the most efficient way of obtaining a trade-off
            between performance and complexity. Nd = Na is a reasonable choice for most datasets. A
            very high value of Nd and Na may suffer from overfitting and yield poor generalization.
            - An optimal choice of γ can have a major role on the overall performance. Typically a larger
            Nsteps value favors for a larger γ.
            - A large batch size is beneficial for performance - if the memory constraints permit, as large
            as 1-10 % of the total training dataset size is suggested. The virtual batch size is typically
            much smaller than the batch size.
            - Initially large learning rate is important, which should be gradually decayed until convergence.
        Args:
            feature_columns: The Tensorflow feature columns for the dataset.
            num_layers: Number of TabNets to stack together.
            feature_dim (N_a): Dimensionality of the hidden representation in feature
                transformation block. Each layer first maps the representation to a
                2*feature_dim-dimensional output and half of it is used to determine the
                nonlinearity of the GLU activation where the other half is used as an
                input to GLU, and eventually feature_dim-dimensional output is
                transferred to the next layer. Can be either a single int, or a list of
                integers. If a list, must be of same length as the number of layers.
            output_dim (N_d): Dimensionality of the outputs of each decision step, which is
                later mapped to the final classification or regression output.
                Can be either a single int, or a list of
                integers. If a list, must be of same length as the number of layers.
            num_features: The number of input features (i.e the number of columns for
                tabular data assuming each feature is represented with 1 dimension).
            num_decision_steps(N_steps): Number of sequential decision steps.
            relaxation_factor (gamma): Relaxation factor that promotes the reuse of each
                feature at different decision steps. When it is 1, a feature is enforced
                to be used only at one decision step and as it increases, more
                flexibility is provided to use a feature at multiple decision steps.
            sparsity_coefficient (lambda_sparse): Strength of the sparsity regularization.
                Sparsity may provide a favorable inductive bias for convergence to
                higher accuracy for some datasets where most of the input features are redundant.
            norm_type: Type of normalization to perform for the model. Can be either
                'batch' or 'group'. 'group' is the default.
            batch_momentum: Momentum in ghost batch normalization.
            virtual_batch_size: Virtual batch size in ghost batch normalization. The
                overall batch size should be an integer multiple of virtual_batch_size.
            num_groups: Number of groups used for group normalization.
            epsilon: A small number for numerical stability of the entropy calculations.
        """
        super(StackedTabNet, self).__init__(**kwargs)

        if num_layers < 1:
            raise ValueError("`num_layers` cannot be less than 1")

        if type(feature_dim) not in [list, tuple]:
            feature_dim = [feature_dim] * num_layers

        if type(output_dim) not in [list, tuple]:
            output_dim = [output_dim] * num_layers

        if len(feature_dim) != num_layers:
            raise ValueError("`feature_dim` must be a list of length `num_layers`")

        if len(output_dim) != num_layers:
            raise ValueError("`output_dim` must be a list of length `num_layers`")

        self.num_layers = num_layers

        layers = []
        layers.append(
            TabNet(
                feature_columns=feature_columns,
                num_features=num_features,
                feature_dim=feature_dim[0],
                output_dim=output_dim[0],
                num_decision_steps=num_decision_steps,
                relaxation_factor=relaxation_factor,
                sparsity_coefficient=sparsity_coefficient,
                norm_type=norm_type,
                batch_momentum=batch_momentum,
                virtual_batch_size=virtual_batch_size,
                num_groups=num_groups,
                epsilon=epsilon,
            )
        )

        for layer_idx in range(1, num_layers):
            layers.append(
                TabNet(
                    feature_columns=None,
                    num_features=output_dim[layer_idx - 1],
                    feature_dim=feature_dim[layer_idx],
                    output_dim=output_dim[layer_idx],
                    num_decision_steps=num_decision_steps,
                    relaxation_factor=relaxation_factor,
                    sparsity_coefficient=sparsity_coefficient,
                    norm_type=norm_type,
                    batch_momentum=batch_momentum,
                    virtual_batch_size=virtual_batch_size,
                    num_groups=num_groups,
                    epsilon=epsilon,
                )
            )

        self.tabnet_layers = layers

    def call(self, inputs, training=None):
        x = self.tabnet_layers[0](inputs, training=training)

        for layer_idx in range(1, self.num_layers):
            x = self.tabnet_layers[layer_idx](x, training=training)

        return x

    @property
    def tabnets(self):
        return self.tabnet_layers

    @property
    def feature_selection_masks(self):
        return [tabnet.feature_selection_masks for tabnet in self.tabnet_layers]

    @property
    def aggregate_feature_selection_mask(self):
        return [tabnet.aggregate_feature_selection_mask for tabnet in self.tabnet_layers]

In [37]:
class StackedTabNetClassifier(tf.keras.Model):
    def __init__(
        self,
        feature_columns,
        num_classes,
        num_layers=1,
        feature_dim=64,
        output_dim=64,
        num_features=None,
        num_decision_steps=5,
        relaxation_factor=1.5,
        sparsity_coefficient=1e-5,
        norm_type="group",
        batch_momentum=0.98,
        virtual_batch_size=None,
        num_groups=2,
        epsilon=1e-5,
        multi_label=False,
        **kwargs
    ):
        """
        Tensorflow 2.0 implementation of [TabNet: Attentive Interpretable Tabular Learning](https://arxiv.org/abs/1908.07442)
        Stacked variant of the TabNet model, which stacks multiple TabNets into a singular model.
        # Hyper Parameter Tuning (Excerpt from the paper)
        We consider datasets ranging from ∼10K to ∼10M training points, with varying degrees of fitting
        difficulty. TabNet obtains high performance for all with a few general principles on hyperparameter
        selection:
            - Most datasets yield the best results for Nsteps ∈ [3, 10]. Typically, larger datasets and
            more complex tasks require a larger Nsteps. A very high value of Nsteps may suffer from
            overfitting and yield poor generalization.
            - Adjustment of the values of Nd and Na is the most efficient way of obtaining a trade-off
            between performance and complexity. Nd = Na is a reasonable choice for most datasets. A
            very high value of Nd and Na may suffer from overfitting and yield poor generalization.
            - An optimal choice of γ can have a major role on the overall performance. Typically a larger
            Nsteps value favors for a larger γ.
            - A large batch size is beneficial for performance - if the memory constraints permit, as large
            as 1-10 % of the total training dataset size is suggested. The virtual batch size is typically
            much smaller than the batch size.
            - Initially large learning rate is important, which should be gradually decayed until convergence.
        Args:
            feature_columns: The Tensorflow feature columns for the dataset.
            num_classes: Number of classes.
            num_layers: Number of TabNets to stack together.
            feature_dim (N_a): Dimensionality of the hidden representation in feature
                transformation block. Each layer first maps the representation to a
                2*feature_dim-dimensional output and half of it is used to determine the
                nonlinearity of the GLU activation where the other half is used as an
                input to GLU, and eventually feature_dim-dimensional output is
                transferred to the next layer. Can be either a single int, or a list of
                integers. If a list, must be of same length as the number of layers.
            output_dim (N_d): Dimensionality of the outputs of each decision step, which is
                later mapped to the final classification or regression output.
                Can be either a single int, or a list of
                integers. If a list, must be of same length as the number of layers.
            num_features: The number of input features (i.e the number of columns for
                tabular data assuming each feature is represented with 1 dimension).
            num_decision_steps(N_steps): Number of sequential decision steps.
            relaxation_factor (gamma): Relaxation factor that promotes the reuse of each
                feature at different decision steps. When it is 1, a feature is enforced
                to be used only at one decision step and as it increases, more
                flexibility is provided to use a feature at multiple decision steps.
            sparsity_coefficient (lambda_sparse): Strength of the sparsity regularization.
                Sparsity may provide a favorable inductive bias for convergence to
                higher accuracy for some datasets where most of the input features are redundant.
            norm_type: Type of normalization to perform for the model. Can be either
                'batch' or 'group'. 'group' is the default.
            batch_momentum: Momentum in ghost batch normalization.
            virtual_batch_size: Virtual batch size in ghost batch normalization. The
                overall batch size should be an integer multiple of virtual_batch_size.
            num_groups: Number of groups used for group normalization.
            epsilon: A small number for numerical stability of the entropy calculations.
        """
        super(StackedTabNetClassifier, self).__init__(**kwargs)

        self.num_classes = num_classes

        self.stacked_tabnet = StackedTabNet(
            feature_columns=feature_columns,
            num_layers=num_layers,
            feature_dim=feature_dim,
            output_dim=output_dim,
            num_features=num_features,
            num_decision_steps=num_decision_steps,
            relaxation_factor=relaxation_factor,
            sparsity_coefficient=sparsity_coefficient,
            norm_type=norm_type,
            batch_momentum=batch_momentum,
            virtual_batch_size=virtual_batch_size,
            num_groups=num_groups,
            epsilon=epsilon,
        )
        if multi_label:

            self.clf = tf.keras.layers.Dense(num_classes, activation="sigmoid", use_bias=False)

        else:

            self.clf = tf.keras.layers.Dense(num_classes, activation="softmax", use_bias=False)

    def call(self, inputs, training=None):
        self.activations = self.stacked_tabnet(inputs, training=training)
        out = self.clf(self.activations)

        return out

In [38]:
class StackedTabNetRegressor(tf.keras.Model):
    def __init__(
        self,
        feature_columns,
        num_regressors,
        num_layers=1,
        feature_dim=64,
        output_dim=64,
        num_features=None,
        num_decision_steps=5,
        relaxation_factor=1.5,
        sparsity_coefficient=1e-5,
        norm_type="group",
        batch_momentum=0.98,
        virtual_batch_size=None,
        num_groups=2,
        epsilon=1e-5,
        **kwargs
    ):
        """
        Tensorflow 2.0 implementation of [TabNet: Attentive Interpretable Tabular Learning](https://arxiv.org/abs/1908.07442)
        Stacked variant of the TabNet model, which stacks multiple TabNets into a singular model.
        # Hyper Parameter Tuning (Excerpt from the paper)
        We consider datasets ranging from ∼10K to ∼10M training points, with varying degrees of fitting
        difficulty. TabNet obtains high performance for all with a few general principles on hyperparameter
        selection:
            - Most datasets yield the best results for Nsteps ∈ [3, 10]. Typically, larger datasets and
            more complex tasks require a larger Nsteps. A very high value of Nsteps may suffer from
            overfitting and yield poor generalization.
            - Adjustment of the values of Nd and Na is the most efficient way of obtaining a trade-off
            between performance and complexity. Nd = Na is a reasonable choice for most datasets. A
            very high value of Nd and Na may suffer from overfitting and yield poor generalization.
            - An optimal choice of γ can have a major role on the overall performance. Typically a larger
            Nsteps value favors for a larger γ.
            - A large batch size is beneficial for performance - if the memory constraints permit, as large
            as 1-10 % of the total training dataset size is suggested. The virtual batch size is typically
            much smaller than the batch size.
            - Initially large learning rate is important, which should be gradually decayed until convergence.
        Args:
            feature_columns: The Tensorflow feature columns for the dataset.
            num_regressors: Number of regressors.
            num_layers: Number of TabNets to stack together.
            feature_dim (N_a): Dimensionality of the hidden representation in feature
                transformation block. Each layer first maps the representation to a
                2*feature_dim-dimensional output and half of it is used to determine the
                nonlinearity of the GLU activation where the other half is used as an
                input to GLU, and eventually feature_dim-dimensional output is
                transferred to the next layer. Can be either a single int, or a list of
                integers. If a list, must be of same length as the number of layers.
            output_dim (N_d): Dimensionality of the outputs of each decision step, which is
                later mapped to the final classification or regression output.
                Can be either a single int, or a list of
                integers. If a list, must be of same length as the number of layers.
            num_features: The number of input features (i.e the number of columns for
                tabular data assuming each feature is represented with 1 dimension).
            num_decision_steps(N_steps): Number of sequential decision steps.
            relaxation_factor (gamma): Relaxation factor that promotes the reuse of each
                feature at different decision steps. When it is 1, a feature is enforced
                to be used only at one decision step and as it increases, more
                flexibility is provided to use a feature at multiple decision steps.
            sparsity_coefficient (lambda_sparse): Strength of the sparsity regularization.
                Sparsity may provide a favorable inductive bias for convergence to
                higher accuracy for some datasets where most of the input features are redundant.
            norm_type: Type of normalization to perform for the model. Can be either
                'batch' or 'group'. 'group' is the default.
            batch_momentum: Momentum in ghost batch normalization.
            virtual_batch_size: Virtual batch size in ghost batch normalization. The
                overall batch size should be an integer multiple of virtual_batch_size.
            num_groups: Number of groups used for group normalization.
            epsilon: A small number for numerical stability of the entropy calculations.
        """
        super(StackedTabNetRegressor, self).__init__(**kwargs)

        self.num_regressors = num_regressors

        self.stacked_tabnet = StackedTabNet(
            feature_columns=feature_columns,
            num_layers=num_layers,
            feature_dim=feature_dim,
            output_dim=output_dim,
            num_features=num_features,
            num_decision_steps=num_decision_steps,
            relaxation_factor=relaxation_factor,
            sparsity_coefficient=sparsity_coefficient,
            norm_type=norm_type,
            batch_momentum=batch_momentum,
            virtual_batch_size=virtual_batch_size,
            num_groups=num_groups,
            epsilon=epsilon,
        )

        self.regressor = tf.keras.layers.Dense(num_regressors, use_bias=False)

    def call(self, inputs, training=None):
        self.activations = self.tabnet(inputs, training=training)
        out = self.regressor(self.activations)
        return outl

## 学習

In [20]:
N_STARTS = 1
N_SPLITS = 7

res = target_df.copy()
submit_df.loc[:, target_df.columns] = 0
res.loc[:, target_df.columns] = 0

for seed in range(N_STARTS):
    for n, (tr, te) in enumerate(
        MultilabelStratifiedKFold(n_splits=N_SPLITS, random_state=seed, shuffle=True).split(target_df, target_df)
    ):
        print("")
        print(f"===== Repeat: {seed}, Fold: {n} =====")

        model = create_model()
        reduce_lr_loss = ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=2, verbose=1, mode="min")
        early_stopping = EarlyStopping(
            monitor="val_loss",
            patience=5,
            mode="min",
            verbose=1,
            restore_best_weights=True,
        )
        model.fit(
            train.values[tr],
            target_df.values[tr],
            validation_data=(train.values[te], target_df.values[te]),
            epochs=50,
            batch_size=128,
            callbacks=[reduce_lr_loss, early_stopping],
            verbose=1,
        )

        test_predict = model.predict(test.values)
        val_predict = model.predict(train.values[te])

        submit_df.loc[:, target_df.columns] += test_predict
        res.loc[te, target_df.columns] += val_predict

submit_df.loc[:, target_df.columns] /= (n + 1) * N_STARTS
res.loc[:, target_df.columns] /= N_STARTS

/home/ubuntu/work/tensorflow-gpu/.venv/lib/python3.6/site-packages/sklearn/utils/validation.py:71: FutureWarning: Pass shuffle=True, random_state=0 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)



===== Repeat: 0, Fold: 0 =====
Epoch 1/50
147/147 [==============================] - 4s 26ms/step - loss: 0.5209 - val_loss: 0.1630
Epoch 2/50
147/147 [==============================] - 3s 23ms/step - loss: 0.0826 - val_loss: 0.0400
Epoch 3/50
147/147 [==============================] - 3s 24ms/step - loss: 0.0319 - val_loss: 0.0253
Epoch 4/50
147/147 [==============================] - 3s 24ms/step - loss: 0.0241 - val_loss: 0.0215
Epoch 5/50
147/147 [==============================] - 3s 23ms/step - loss: 0.0216 - val_loss: 0.0200
Epoch 6/50
147/147 [==============================] - 3s 24ms/step - loss: 0.0199 - val_loss: 0.0191
Epoch 7/50
147/147 [==============================] - 3s 23ms/step - loss: 0.0189 - val_loss: 0.0184
Epoch 8/50
147/147 [==============================] - 3s 23ms/step - loss: 0.0181 - val_loss: 0.0181
Epoch 9/50
147/147 [==============================] - 3s 23ms/step - loss: 0.0176 - val_loss: 0.0176
Epoch 10/50
147/147 [==============================] - 3s 2

## Cross Validation

In [21]:
target_cols = target_df.columns.values.tolist()

y_true = target_df.values
y_pred = res.values

score = 0
for i in range(len(target_cols)):
    score_ = log_loss(y_true[:, i], y_pred[:, i])
    score += score_ / target_df.shape[1]

print("CV log_loss: ", score)

CV log_loss:  0.016486476250573326


## 補正

In [22]:
submit_df.loc[test_df["cp_type"] == "ctl_vehicle", target_df.columns] = 0

## t検定

https://logics-of-blue.com/t-test/

学習データとテストデータに有意な差があるか。

p値が0.05を下回ると、有意な差があると言える。


In [23]:
"""
g-0 0.032543038217202086
g-1 0.019393109454476025
g-3 0.046721309178349664
g-22 6.587624616736739e-05
g-37 0.010911040292799237
g-48 0.030801708455925808
g-50 0.03531252898048682
g-52 0.028092336066332588
g-60 0.003985876456805001
g-72 0.03734120596180097
g-98 0.002596791303651995
g-100 0.008351686077946879
g-101 3.938347554051903e-05
g-105 0.0429096270660344
g-110 0.007002341005554218
g-119 0.0026784043039369777
g-120 0.04036243989718826
g-121 0.018277391157227284
g-134 0.024952122240263698
g-135 5.2762961694835665e-05
g-136 0.014781889784802717
g-139 0.032409758453034926
g-140 0.029888796018104586
g-145 0.04082269273028868
g-150 0.02741917010243746
g-152 0.029540491011946193
g-158 0.006359327029617653
g-165 0.015874095698013277
g-166 0.044642334463927895
g-168 0.04202225455856443
g-169 0.03499856643946845
g-174 0.018682872175197398
g-186 0.008947447524514207
g-193 0.027903058327269626
g-206 0.031924304036266206
g-211 0.0011897547708701196
g-215 0.023199568324813253
g-224 0.03730567820677584
g-235 0.03499601003997585
g-237 0.023174582508069416
g-240 0.04288696473409333
g-248 0.031831549496090336
g-257 0.0006119796151287299
g-267 0.04980450859091937
g-273 0.006297806993275883
g-284 0.04655082243178294
g-298 0.015033343937168856
g-322 0.01744507921578457
g-330 0.011666755951211137
g-332 0.0021244787311601766
g-343 0.03324481374310012
g-346 0.01355309132510231
g-354 0.0016517824313901726
g-360 0.009620025505946095
g-362 0.0324817698501945
g-365 0.012689368176776432
g-367 0.023754819659996564
g-378 0.04656608215292139
g-380 0.0017078489673201087
g-386 0.009849666373976277
g-390 0.03601825397540972
g-392 0.00014622041338638775
g-396 0.011558383816503901
g-400 0.00234417466908514
g-406 0.03060881366633618
g-410 0.003942306812742894
g-413 0.008357431453948386
g-418 0.002296074513896323
g-433 0.0028952219485856396
g-437 0.03037716243553023
g-439 0.012787195107598747
g-440 0.0009790120173915739
g-445 0.03942676587611877
g-451 0.03521936791875377
g-461 0.026308390115405485
g-473 0.017296065430318728
g-485 0.04687073341670321
g-486 0.013919900233126406
g-489 0.0010059622726513917
g-491 0.0029324527830305452
g-492 0.00048123771140560774
g-498 0.012101192831340432
g-503 0.0015522760171656617
g-518 0.0160779468086828
g-526 0.006223261683449866
g-528 0.01243903801888239
g-554 0.005526846061505236
g-568 0.03981859366092907
g-574 0.0009977307203055188
g-577 0.017459489508530397
g-586 0.024064208550529632
g-600 0.002201740921036897
g-619 0.016276470302135262
g-628 0.0013193596507959106
g-655 0.01974989210697146
g-656 0.0001817267948382975
g-664 0.005051744552255386
g-672 0.03407566901525629
g-675 0.011914459074866945
g-683 0.03929606627209026
g-700 0.0037311306453646255
g-702 0.0010125897831886554
g-708 0.012301558853143693
g-709 0.0398328066319844
g-719 0.04875188668684708
g-723 0.029252768384844995
g-725 0.026614623630592568
g-743 0.040185681769053454
g-744 0.036928324629596374
g-745 0.018719801905843145
g-757 0.04915580418862598
g-761 0.005939948820549039
g-767 0.011931339152979633
g-770 0.010495387864363378
g-771 0.02066501430630159
c-9 0.00676768179495497
c-10 0.027463106489330136
c-18 0.027953170579559945
c-22 0.012676157706292834
c-49 0.04890393979396845
c-70 0.03525132105881509
c-97 0.03324866397008827
c-98 0.03673769845220116

合計: 123
"""

'\ng-0 0.032543038217202086\ng-1 0.019393109454476025\ng-3 0.046721309178349664\ng-22 6.587624616736739e-05\ng-37 0.010911040292799237\ng-48 0.030801708455925808\ng-50 0.03531252898048682\ng-52 0.028092336066332588\ng-60 0.003985876456805001\ng-72 0.03734120596180097\ng-98 0.002596791303651995\ng-100 0.008351686077946879\ng-101 3.938347554051903e-05\ng-105 0.0429096270660344\ng-110 0.007002341005554218\ng-119 0.0026784043039369777\ng-120 0.04036243989718826\ng-121 0.018277391157227284\ng-134 0.024952122240263698\ng-135 5.2762961694835665e-05\ng-136 0.014781889784802717\ng-139 0.032409758453034926\ng-140 0.029888796018104586\ng-145 0.04082269273028868\ng-150 0.02741917010243746\ng-152 0.029540491011946193\ng-158 0.006359327029617653\ng-165 0.015874095698013277\ng-166 0.044642334463927895\ng-168 0.04202225455856443\ng-169 0.03499856643946845\ng-174 0.018682872175197398\ng-186 0.008947447524514207\ng-193 0.027903058327269626\ng-206 0.031924304036266206\ng-211 0.0011897547708701196\ng-215 

In [24]:
submit_df.to_csv("submission.csv", index=False)